In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense



In [11]:
max_features = 10000
(X_train,y_train),(X_test,y_test) = imdb.load_data(num_words = max_features)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:
max_len = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)

AttributeError: module 'keras._tf_keras.keras.preprocessing.sequence' has no attribute 'pad_sequence'

In [ ]:
model = Sequential()
model.add(Embedding(max_features,128,input_length = max_len))
model.add(SimpleRNN(128,activation='relu'))
model.add(Dense(1,activation ='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
earlystopping = EarlyStopping(monitor='val_loss',patience =5,restore_best_weights =True)

In [ ]:
history = model.fit(
    X_train,y_train,epochs=15,batch_size =32,
    validation_split=0.2,
    callbacks=[earlystopping]
)

In [24]:
model_path = 'simple_rnn_imdb.h5'
# load full saved model
model = tf.keras.models.load_model(model_path)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [ ]:
# model.get_weights() removed - not needed for this walkthrough

[array([[-0.22785607,  0.24449548,  0.1923425 , ...,  0.29184812,
          0.29042685,  0.18941468],
        [-0.00642405,  0.02297622, -0.05032053, ...,  0.01828327,
          0.01101572,  0.02337427],
        [ 0.04475113,  0.07341534, -0.04180979, ..., -0.0688847 ,
          0.1087291 ,  0.1282114 ],
        ...,
        [ 0.00162585, -0.01890367,  0.10140095, ...,  0.10429546,
         -0.01117515, -0.02471659],
        [ 0.05718635, -0.0555069 ,  0.02457928, ..., -0.05139313,
         -0.03350348, -0.0608652 ],
        [ 0.0441264 , -0.03814809,  0.07965294, ...,  0.00652633,
         -0.00371443, -0.06140587]], shape=(10000, 128), dtype=float32),
 array([[-0.00597223, -0.02244808,  0.02256939, ...,  0.13137603,
          0.14134172,  0.06965625],
        [-0.05559777, -0.0446468 , -0.13881561, ..., -0.15772684,
          0.05440029, -0.16952917],
        [ 0.1521306 , -0.12868953, -0.09378973, ...,  0.03372649,
          0.01279084, -0.14808318],
        ...,
        [-0.0669046

In [25]:
from tensorflow.keras.preprocessing import sequence

In [28]:
# create word index mappings (uses imdb imported earlier)
word_index = imdb.get_word_index()
reverse_word_index = {v: k for k, v in word_index.items()}

def decode_review(encoded_review):
    """
    Decode an encoded review (list/array of ints) back to a string.
    Uses the same indexing offset used by the Keras IMDB loader (reserved 0-3).
    """
    return ' '.join(reverse_word_index.get(i - 3, '?') for i in encoded_review)

def preprocess_text(text):
    """
    Convert a raw text string into a padded integer sequence suitable for the model.
    - unknown words map to index 2 (unknown) then offset by 3
    - words with indices >= max_features are treated as unknown
    - returns a 2D numpy array of shape (1, max_len)
    """
    words = text.lower().split()
    encoded = []
    for w in words:
        raw_idx = word_index.get(w, None)
        # If word not found or index outside the model's vocabulary, use unknown token (2).
        if raw_idx is None or raw_idx >= max_features:
            encoded.append(2)
        else:
            encoded.append(raw_idx + 3)
    padded_review = sequence.pad_sequences([encoded], maxlen=max_len)
    return padded_review

In [29]:
example = "I like chilly fries"
sentiment, score = predict_sentiment(example)
print(f"Sentiment: {sentiment}")
print(f"Prediction Score: {score}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Sentiment: Positive
Prediction Score: 0.9999943375587463
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Sentiment: Positive
Prediction Score: 0.9999943375587463
